In [1]:
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)

class FinanceStat:
    """The model scapes cafef.vn to get 3 finanicial statement about a company.
    Params:
        company_name: name on stock market
    
    Function:
        self.get_findata(report_type):
            get finance data according to the report
        self.export_findata(form='csv'):
            export data to csv or xlsx format
    """
    # input
    def __init__(self, company_name='fpt'):
        self.company_name = company_name
        # internal variable
        self.year_1st = 2015
        self.year_final = datetime.datetime.now().year
    
    def get_category(self, report_type):
        url = ('https://s.cafef.vn/bao-cao-tai-chinh/' 
               + self.company_name + '/' 
               + report_type + '/' 
               + '2024' + 
               '/0/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn'
               )  # pick a random year to get category
        df = pd.read_html(url)
        df_temp = df[4]
        df_category = df_temp.iloc[:, 0]
        return df_category
        
    def get_stat(self, report_type):
        
        year_list = range(self.year_1st, self.year_final+1)
        
        result = pd.DataFrame(columns = year_list)
        
        for year in year_list:
            url = ('https://s.cafef.vn/bao-cao-tai-chinh/' 
                    + self.company_name + '/' 
                    + report_type + '/' 
                    + str(year) + 
                    '/0/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn'
                    )
            
            web_data = pd.read_html(url)

            # construct dataframe from data         
            result[year] = web_data[4].iloc[:, 4]
            
        
            # check if data available or not
            if result[year].isna().sum() == len(result):
                result.drop(columns=year, inplace=True)
        
        return result
    
    def get_findata(self, report_type, report_format='analysis'):
        """
        Parameter:
            report_type: ['cashflow', 'incsta', 'bsheet']
            report_format: 'analysis', 'traditional'
        """
        result = pd.concat([self.get_category(report_type),
                          self.get_stat(report_type)],
                         axis='columns')
        result.set_index(0, inplace=True)
        
        if report_format == 'analysis':
            return result.T
        else:
            return result
    
    def export_findata(self, form='csv', report_format='analysis'):
        """
        Parameters:
            form: ['csv', 'xlsx']
            report_format: ['analysis', 'traditional']
        """
        for report_type in ['cashflow', 'incsta', 'bsheet']:
            file_name = self.company_name + '_' + report_type
            if form == 'csv':
                self.get_findata(report_type, report_format).to_csv(file_name + '.csv')
            else:
                self.get_findata(report_type, report_format).to_excel(file_name + '.xlsx')

### Get Company's Stats from Vietcap

In [2]:
from vnstock.explorer.vci import Listing, Quote, Company, Finance, Trading
# VCI là nguồn dữ liệu từ CK Vietcap bên cạnh nguồn từ TCBS
from vnstock import Listing
listing = Listing()
df_list_symbols_by_exchange = listing.symbols_by_exchange()
df_stock_short = df_list_symbols_by_exchange[
    df_list_symbols_by_exchange['exchange'].isin(['HSX', 'HNX'])
    & (df_list_symbols_by_exchange['type'] == 'STOCK')
]

df_symbols_by_industries = listing.symbols_by_industries()
df_stock_industry = df_symbols_by_industries[['symbol', 'icb_name2']].rename(columns = {'icb_name2': 'industry'})
df_stock_short = pd.merge(df_stock_short, df_stock_industry, how = 'left', on = 'symbol')

In [3]:
df_stock_short.head(10)

,symbol,exchange,type,organ_short_name,organ_name,product_grp_id,industry
0,YEG,HSX,STOCK,Tập đoàn Yeah1,Công ty Cổ phần Tập đoàn Yeah1,STO,Truyền thông
1,YBM,HSX,STOCK,Khoáng sản CN Yên Bái,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,STO,Tài nguyên Cơ bản
2,X20,HNX,STOCK,May mặc X20,Công ty Cổ phần X20,STX,Hàng cá nhân & Gia dụng
3,WSS,HNX,STOCK,Chứng khoán Phố Wall,Công ty Cổ phần Chứng khoán Phố Wall,STX,Dịch vụ tài chính
4,WCS,HNX,STOCK,Bến xe Miền Tây,Công ty Cổ phần Bến xe Miền Tây,STX,Hàng & Dịch vụ Công nghiệp
5,VTZ,HNX,STOCK,Nhựa Việt Thành,Công ty Cổ phần Sản xuất và Thương mại Nhựa Vi...,STX,Hóa chất
6,VTV,HNX,STOCK,Năng lượng và Môi trường VICEM,Công ty Cổ phần Năng lượng và Môi trường VICEM,STX,Xây dựng và Vật liệu
7,VTP,HSX,STOCK,Bưu chính Viettel,Tổng Công ty Cổ phần Bưu chính Viettel,STO,Hàng & Dịch vụ Công nghiệp
8,VTO,HSX,STOCK,VITACO,Công ty Cổ phần Vận tải Xăng dầu VITACO,STO,Hàng & Dịch vụ Công nghiệp
9,VTJ,HNX,STOCK,TM và Đầu tư VINATABA,Công ty Cổ phần Thương mại và Đầu tư VI NA TA BA,STX,Hàng cá nhân & Gia dụng


In [22]:
df_stock_short.to_excel("Vietcap__Company_List.xlsx")

### Get Company's FS from CafeF

In [4]:
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import os # To help determine a good number of workers

# This function contains the logic from your original for loop.
def process_company(company_nm, stock_info_df):
    """
    Fetches and processes all financial reports for a single company.
    """
    try:
        model = FinanceStat(company_name=company_nm)
        list_report_types = ['bsheet', 'incsta', 'cashflow']
        list_report_dfs = []

        for report_i in list_report_types:
            try:
                report_df = model.get_findata(report_i)
                report_df.reset_index(inplace=True)

                # Melt the dataframe
                df_melted = pd.melt(
                    report_df,
                    id_vars=['index'],
                    var_name='account',
                    value_name='value'
                )

                df_melted['report_date'] = df_melted['index'].astype(str)
                df_melted['symbol'] = company_nm

                # Clean and filter
                df_cleaned = df_melted[['symbol', 'report_date', 'account', 'value']]
                df_cleaned = df_cleaned[df_cleaned['report_date'] >= '2000'].copy()

                # Merge with stock info
                df_report_join = pd.merge(df_cleaned, stock_info_df, how='left', on='symbol')

                # Assign report type
                report_type_map = {
                    'bsheet': 'Balance Sheet',
                    'incsta': 'Income Statement',
                    'cashflow': 'Cash Flow Statement'
                }
                df_report_join['report_type'] = report_type_map.get(report_i)

                # Final column selection and sorting
                final_cols = ['symbol', 'exchange', 'organ_name', 'industry', 'report_type', 'report_date', 'account', 'value']
                df_report_join = df_report_join[final_cols]
                
                list_report_dfs.append(df_report_join)

            except Exception as e:
                # This error is for a single report (e.g., 'bsheet') failing
                print(f"ERROR processing report '{report_i}' for company '{company_nm}': {e}")
                continue # Continue to the next report type

        if not list_report_dfs:
            return None # Return None if no reports were successfully processed for the company

        # Concatenate all reports for the company
        company_financial_report = pd.concat(list_report_dfs, axis=0, ignore_index=True)
        company_financial_report.sort_values(by=['symbol', 'report_date'], inplace=True)
        return company_financial_report

    except Exception as e:
        # This error is for the entire company failing (e.g., invalid symbol)
        print(f"FATAL ERROR processing company '{company_nm}': {e}")
        return None


# --- Step 2: Use ThreadPoolExecutor to run the function in parallel ---
if __name__ == "__main__":
    list_symbol = list(df_stock_short['symbol'])
    # list_symbol = ['VNM', 'FPT', 'HPG', 'TCB', 'ACB', 'BID', 'CTG', 'GVR', 'HDB', 'KDH'] # Example list
    
    final_company_reports = []
    
    # Set the number of threads. A good starting point for I/O tasks is 5 * number of CPU cores.
    # Be mindful of API rate limits!
    MAX_WORKERS = 10 
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Create a future for each company symbol. Pass the stock_info_df to each worker.
        # The `map` function applies `process_company` to each item in `list_symbol`.
        future_to_company = {executor.submit(process_company, sym, df_stock_short): sym for sym in list_symbol}
        
        # Use tqdm to create a progress bar
        for future in tqdm(concurrent.futures.as_completed(future_to_company), total=len(list_symbol), desc="Crawling Financial Data"):
            result_df = future.result()
            if result_df is not None and not result_df.empty:
                final_company_reports.append(result_df)

    # --- Step 3: Concatenate all results into a single DataFrame ---
    if final_company_reports:
        final_df = pd.concat(final_company_reports, ignore_index=True)
        print("\nCrawling complete!")
        print(f"Successfully processed {final_df['symbol'].nunique()} companies.")
        print(final_df.head())
    else:
        print("\nCrawling complete, but no data was retrieved.")

Crawling Financial Data:   6%|███▏                                                    | 39/697 [02:50<16:25,  1.50s/it]

ERROR processing report 'cashflow' for company 'VNR': HTTP Error 404: Not Found


Crawling Financial Data:  37%|███████████████████▊                                 | 260/697 [19:47<1:02:21,  8.56s/it]

ERROR processing report 'cashflow' for company 'PTI': HTTP Error 404: Not Found


Crawling Financial Data:  39%|█████████████████████▏                                 | 269/697 [20:23<35:33,  4.98s/it]

...

ERROR processing report 'cashflow' for company 'BMI': HTTP Error 404: Not Found


Crawling Financial Data:  94%|███████████████████████████████████████████████████▌   | 654/697 [47:56<01:16,  1.79s/it]

ERROR processing report 'cashflow' for company 'BIC': HTTP Error 404: Not Found


Crawling Financial Data: 100%|███████████████████████████████████████████████████████| 697/697 [50:44<00:00,  4.37s/it]



Crawling complete!
Successfully processed 696 companies.
  symbol exchange                       organ_name  \
0    WCS      HNX  Công ty Cổ phần Bến xe Miền Tây   
1    WCS      HNX  Công ty Cổ phần Bến xe Miền Tây   
2    WCS      HNX  Công ty Cổ phần Bến xe Miền Tây   
3    WCS      HNX  Công ty Cổ phần Bến xe Miền Tây   
4    WCS      HNX  Công ty Cổ phần Bến xe Miền Tây   

                     industry    report_type report_date  \
0  Hàng & Dịch vụ Công nghiệp  Balance Sheet        2015   
1  Hàng & Dịch vụ Công nghiệp  Balance Sheet        2015   
2  Hàng & Dịch vụ Công nghiệp  Balance Sheet        2015   
3  Hàng & Dịch vụ Công nghiệp  Balance Sheet        2015   
4  Hàng & Dịch vụ Công nghiệp  Balance Sheet        2015   

                                 account           value  
0                                TÀI SẢN             NaN  
1                    A- TÀI SẢN NGẮN HẠN  164531399389.0  
2  I. Tiền và các khoản tương đương tiền   35397973902.0  
3                   

In [10]:
final_df.to_csv('CafeF__Finance_Statement__20250813.csv', sep="\t")

In [15]:
import json
def load_translation_dictionary(file_path: str) -> dict:
    with open(file_path, 'r', encoding='utf-8') as f:
        translation_dict = json.load(f)
    print(f"Tải thành công dictionary từ '{file_path}'")
    return translation_dict

file_name = 'account_mapping_format.json'
account_dictionary = load_translation_dictionary(file_name)

Tải thành công dictionary từ 'account_mapping_format.json'


In [46]:
# df_financial_report = pd.concat(final_company_report, axis = 0)
financial_statement_schemas = ['company_code', 'exchange', 'company_name', 'industry', 'report_type', 'report_date', 'account', 'value', 'account_vi', 'account_en']
df_financial_report = final_df.copy()
df_financial_report['account_vi'] = df_financial_report['account'] 
df_financial_report['account_en'] = df_financial_report['account'].apply(lambda x: account_dictionary[x]['english'] if x in account_dictionary.keys() else '')
df_financial_report['account'] = df_financial_report['account'].apply(lambda x: account_dictionary[x]['english_format'] if x in account_dictionary.keys() else '')
df_financial_report.columns = financial_statement_schemas

### Thống kê `account` bị NaN

In [47]:
df_alias_miss = df_financial_report[df_financial_report['account'] == '']
df_alias_miss.drop_duplicates(['company_code', 'report_date', 'report_type'], inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [49]:
set(df_alias_miss.account_vi)

{'1- Thu phí bảo hiểm gốc', '2. 2. Đầu tư ngắn hạn khác'}

In [45]:
len(list_company_miss)

8

### Save Adhoc

In [54]:
df_financial_report_sample

,company_code,exchange,company_name,industry,report_type,report_date,account,value,account_vi,account_en
943835,CMG,HSX,Công ty Cổ phần Tập đoàn Công nghệ CMC,Công nghệ Thông tin,Balance Sheet,2021,intragroup_payables_capital,NaN,4. Phải trả nội bộ về vốn kinh doanh,4. Intra-group payables on capital
705516,HTI,HSX,Công ty Cổ phần Đầu tư Phát triển Hạ tầng IDICO,Xây dựng và Vật liệu,Balance Sheet,2022,inventories,99348958.0,IV. Hàng tồn kho,IV. Inventories
692452,HVN,HSX,Tổng Công ty Hàng không Việt Nam - CTCP,Du lịch và Giải trí,Cash Flow Statement,2021,payments_other_operating,-317700707449.0,- Tiền chi khác cho hoạt động kinh doanh,- Other payments for operating activities
286148,ST8,HSX,Công ty Cổ phần Tập đoàn ST8,Công nghệ Thông tin,Income Statement,2023,diluted_eps,NaN,22. Lãi suy giảm trên cổ phiếu (*),22. Diluted earnings per share (*)
37864,VPD,HSX,Công ty Cổ phần Phát triển Điện lực Việt Nam,"Điện, nước & xăng dầu khí đốt",Cash Flow Statement,2021,tax_paid,NaN,- Thuế thu nhập doanh nghiệp đã nộp,- Income tax paid
...,...,...,...,...,...,...,...,...,...,...
642692,KSD,HNX,Công ty Cổ phần Đầu tư DNA,Hàng cá nhân & Gia dụng,Balance Sheet,2022,htm_inv_st,NaN,3. Đầu tư nắm giữ đến ngày đáo hạn,3. Held-to-maturity investments
168217,TTF,HSX,Công ty Cổ phần Tập đoàn Kỹ nghệ Gỗ Trường Thành,Tài nguyên Cơ bản,Balance Sheet,2019,ded_vat,26204018428.0,2. Thuế GTGT được khấu trừ,2. Deductible value added tax
544259,NFC,HNX,Công ty Cổ phần Phân lân Ninh Bình,Hóa chất,Balance Sheet,2023,ar_cust_lt,NaN,1. Phải thu dài hạn của khách hàng,1. Accounts receivable from customers
808467,FDC,HSX,Công ty Cổ phần Ngoại thương và Phát triển Đầu...,Bất động sản,Cash Flow Statement,2016,operating_profit_before_wc_changes,-3270332611.0,3. Lợi nhuận từ hoạt động kinh doanh trước tha...,3. Operating profit before changes in working ...


In [53]:
df_financial_report_sample = df_financial_report.sample(100)
df_financial_report_sample.to_csv('Financial_Statement__Full_Company_sample.csv', sep = "\t")

In [80]:
df_financial_report_short = df_financial_report[['company_code', 'exchange', 'company_name', 'industry', 'report_type', 'report_date', 'account', 'value']]

In [90]:
df_financial_report.value = df_financial_report.value.astype(float)
df_financial_report.to_parquet('Financial_Statement__Full_Company_L10Y.parquet')

df_lod = pd.read_parquet('Financial_Statement__Full_Company_L10Y.parquet')

In [82]:
df_financial_report_short.to_csv('Financial_Statement__Full_Company_L10Y_2.csv', sep = "\t")

In [75]:
df_financial_report_new = df_financial_report[(df_financial_report['report_date'] <= '2025') &(df_financial_report['report_date'] >= '2017')]

In [76]:
df_financial_report_new.shape

(951175, 10)

In [77]:
df_financial_report.shape

(1089110, 10)

In [78]:
df_financial_report_new.to_excel('Financial_Statement__Full_Company_L10Y.xlsx')

In [29]:
df_financial_report_5y = df_financial_report[df_financial_report['report_date'] >= '2020']

In [30]:
df_financial_report_5y.describe(include='all')

,company_code,exchange,company_name,industry,report_type,report_date,account,value,account_en
count,594642,594642,594642,594642,594642,594642,594642,3.248380e+05,594642
unique,691,2,690,19,3,6,174,2.628020e+05,179
top,BVH,HSX,Công ty Cổ phần Nam Việt,Xây dựng và Vật liệu,Balance Sheet,2021,accum_dep_inv_property,2.000000e+10,- Cost
freq,1122,329536,1756,105165,399174,120794,13224,2.100000e+02,13224


In [32]:
len(set(df_financial_report_5y[df_financial_report_5y['exchange'] == 'HNX'].company_code))

304

In [33]:
len(set(df_financial_report_5y[df_financial_report_5y['exchange'] == 'HSX'].company_code))

387

In [39]:
df_financial_report.account.value_counts()

account
accum_dep_inv_property      24048
inv_property_cost           24048
                            16368
inventories                 12024
pref_shares                 12024
                            ...  
other_receipts_operating     5212
payments_other_operating     5212
net_operating_cash_flows     5212
inv_cash_flows               5212
adj_other_adjustments        5212
Name: count, Length: 174, dtype: int64

In [37]:
df_financial_report.report_type.value_counts()

report_type
Balance Sheet          725988
Cash Flow Statement    218904
Income Statement       144218
Name: count, dtype: int64

In [38]:
df_financial_report_Bsheet = df_financial_report[df_financial_report['report_type'] == 'Balance Sheet']
df_financial_report_Inc = df_financial_report[df_financial_report['report_type'] == 'Income Statement']
df_financial_report_CF = df_financial_report[df_financial_report['report_type'] == 'Cash Flow Statement']

df_financial_report_Bsheet.to_excel('Financial_Statement__Full_Company__BSheet.xlsx')
df_financial_report_Inc.to_excel('Financial_Statement__Full_Company__IncStm.xlsx')
df_financial_report_CF.to_excel('Financial_Statement__Full_Company__CashFlow.xlsx')

In [36]:
df_financial_report_5y.to_excel('Financial_Statement__Full_Company__L5Y.xlsx')

In [60]:
import requests
import time
import certifi # <--- 1. IMPORT CERTIFI

def get_hnx_tickers() -> list[str]:
    """
    Scrapes all stock tickers (Mã CK) listed on the HNX website.

    The function queries the website's underlying API, which provides the data
    in a structured JSON format. It handles pagination to retrieve all entries.

    Returns:
        A list of strings, where each string is a stock ticker.
        Returns an empty list if scraping fails.
    """
    api_url = "https://hnx.vn/vi-vn/cophieu-etfs/chung-khoan-ny.html/search"
    
    # These headers make our request look like it's coming from a real browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
    }

    all_tickers = []
    page = 1
    page_size = 50 # We can request a larger page size to reduce the number of requests

    print("Starting to scrape stock tickers from HNX...")

    while True:
        payload = {
            'p_page': page,
            'p_page_size': page_size,
            'p_key_word': '',
            '_search': 'false',
        }

        try:
            # Make the POST request to the API endpoint
            response = requests.post(api_url, headers=headers, data=payload, timeout=10)
            # This will raise an exception for bad status codes (4xx or 5xx)
            response.raise_for_status() 

            json_data = response.json()
            
            # Extract the list of stock items from the JSON response
            stock_data = json_data.get('data', [])

            # If the 'data' list is empty, it means we've reached the last page
            if not stock_data:
                print("No more data found. Finished scraping.")
                break

            # Use a list comprehension to extract the 'STOCK_CODE' from each item
            tickers_on_page = [item['STOCK_CODE'] for item in stock_data]
            all_tickers.extend(tickers_on_page)

            print(f"Scraped page {page}: Found {len(tickers_on_page)} tickers.")

            # Move to the next page
            page += 1
            
            # Be polite to the server by waiting a moment between requests
            time.sleep(0.5)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred during the request: {e}")
            break
        except KeyError as e:
            print(f"Could not find key {e} in the JSON response. The API structure might have changed.")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break

    print(f"\nTotal tickers found: {len(all_tickers)}")
    return sorted(list(set(all_tickers))) # Return a sorted list with unique values

# --- Example of how to use the function ---
if __name__ == "__main__":
    hnx_stock_list = get_hnx_tickers()
    
    if hnx_stock_list:
        print("\n--- First 20 Tickers ---")
        print(hnx_stock_list[:20])

        print("\n--- Last 20 Tickers ---")
        print(hnx_stock_list[-20:])

Starting to scrape stock tickers from HNX...
An error occurred during the request: HTTPSConnectionPool(host='hnx.vn', port=443): Max retries exceeded with url: /vi-vn/cophieu-etfs/chung-khoan-ny.html/search (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))

Total tickers found: 0


In [57]:
df_financial_report.head(3)

,symbol,exchange,organ_name,industry,report_type,report_date,account,value,account_en
0,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,total_asset,NaN,ASSETS
5,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,total_asset_st,2.966573e+13,A- Current assets
10,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,cash_and_cash_eq,2.111243e+12,I. Cash and cash equivalents


In [25]:
# balance_sheet_account_df = final_company_report[1]
# balance_sheet_account_list = list(balance_sheet_account_df[balance_sheet_account_df['report_date'] == '2024']['account'])

In [26]:
df_financial_report

,company_code,exchange,company_name,industry,report_type,report_date,account,account_en,value
0,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,TÀI SẢN,NaN,ASSETS
5,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,A- TÀI SẢN NGẮN HẠN,2.966573e+13,A- Current assets
10,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,I. Tiền và các khoản tương đương tiền,2.111243e+12,I. Cash and cash equivalents
15,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,1. Tiền,8.638533e+11,1. Cash
20,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Balance Sheet,2020,2. Các khoản tương đương tiền,1.247390e+12,2. Cash equivalents
...,...,...,...,...,...,...,...,...,...
189,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Cash Flow Statement,2024,Lưu chuyển tiền thuần từ hoạt động tài chính,-6.641260e+12,Net cash flows from financing activities
194,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Cash Flow Statement,2024,Lưu chuyển tiền thuần trong kỳ (50 = 20+30+40),-6.944153e+11,Net cash flows during the year (50 = 20 + 30 +...
199,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Cash Flow Statement,2024,Tiền và tương đương tiền đầu kỳ,2.912027e+12,Cash and cash equivalents at the beginning of ...
204,VNM,HSX,Công ty Cổ phần Sữa Việt Nam,Thực phẩm và đồ uống,Cash Flow Statement,2024,Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi...,8.331710e+09,Effect of exchange rate fluctuations on cash a...


In [26]:
df_financial_report.to_excel("CafeF__Financial_Statement__VNM_sample_en.xlsx")

In [6]:
pd.read_html('https://cafef.vn/du-lieu/bao-cao-tai-chinh/fpt/bsheet/2022/1/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn')[3]

,0,1,2,3,4,5
0,Trước Sau,Quý 2- 2021,Quý 3- 2021,Quý 4- 2021,Quý 1- 2022,Tăng trưởng


In [28]:
excel_path = "account_mapping.xlsx"

df = pd.read_excel(excel_path)

In [30]:
df

,Vietnamese,English,Account format
0,A- TÀI SẢN NGẮN HẠN,A- Current assets,total_asset_st
1,I. Tiền và các khoản tương đương tiền,I. Cash and cash equivalents,cash_and_cash_eq
2,1. Tiền,1. Cash,cash
3,2. Các khoản tương đương tiền,2. Cash equivalents,cash_eq
4,II. Các khoản đầu tư tài chính ngắn hạn,II. Short-term financial investments,fin_inv_st
...,...,...,...
178,Lưu chuyển tiền thuần từ hoạt động tài chính,Net cash flows from financing activities,net_fin_cash_flows
179,Lưu chuyển tiền thuần trong kỳ (50 = 20+30+40),Net cash flows during the year (50 = 20 + 30 +...,net_cash_flows
180,Tiền và tương đương tiền đầu kỳ,Cash and cash equivalents at the beginning of ...,cash_and_cash_eq_begin_year
181,Ảnh hưởng của thay đổi tỷ giá hối đoái quy đổi...,Effect of exchange rate fluctuations on cash a...,effect_exchange_rate


In [39]:
dict(row)

{'Vietnamese ': 'A- TÀI SẢN NGẮN HẠN',
 'English': 'A- Current assets',
 'Account format': 'total_asset_st'}

In [52]:
import pandas as pd
import json

def excel_to_custom_json(excel_path: str, output_json_path: str, sheet_name=0):
    try:
        df = pd.read_excel(excel_path, sheet_name=0)
        result_dict = {}
        for index, row in df.iterrows():
            key = dict(row)['Vietnamese ']
            # Tạo dictionary con cho giá trị
            value_1 = dict(row)['English']
            value_2 = dict(row)['Account format']
            value_dict = {
                'english': str(value_1),
                'english_format': str(value_2) 
            }
            result_dict[key] = value_dict

        # Ghi dictionary kết quả ra file JSON
        with open(output_json_path, 'w', encoding='utf-8') as json_file:
            json.dump(result_dict, json_file, ensure_ascii=False, indent=4)

        print(f"Chuyển đổi thành công! File JSON đã được lưu tại: {output_json_path}")

    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file tại '{excel_path}'")
    except Exception as e:
        print(f"Đã xảy ra một lỗi không mong muốn: {e}")
    return result_dict
input_excel_file = 'account_mapping.xlsx'  # <-- Thay bằng tên file Excel của bạn
output_json_file = 'account_mapping_format.json'   # <-- Tên file JSON bạn muốn tạo
account_dict = excel_to_custom_json(input_excel_file, output_json_file)

Chuyển đổi thành công! File JSON đã được lưu tại: account_mapping_format.json


### Refactor

In [18]:
import pandas as pd
import datetime
import logging
import re

# Configure logging to show info messages
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
pd.set_option('display.max_columns', None)

class FinanceStat:
    """
    The model scrapes cafef.vn to get 3 financial statements about a company.
    Parameters:
        company_name (str): The stock ticker symbol of the company (e.g., 'fpt', 'vnm').
        start_year (int): The first year to start scraping data from.
    """

    def __init__(self, company_name='fpt', start_year=2020):
        if not isinstance(company_name, str) or not company_name:
            raise ValueError("Company name must be a non-empty string.")
        self.company_name = company_name.upper() # Standardize to uppercase
        self.start_year = start_year
        self.end_year = datetime.datetime.now().year
        self.report_types = ['incsta', 'bsheet', 'cashflow'] # Income Statement, Balance Sheet, Cash Flow Statement
        
    def _build_url(self, report_type, year, quarter=0):
        """
        Private helper method to construct the correct URL.
        quarter=0 is for yearly reports.
        """
        return (
            'https://s.cafef.vn/bao-cao-tai-chinh/'
            f'{self.company_name}/{report_type}/'
            f'{year}/{quarter}/0/0/0/luu-chuyen-tien-te-gian-tiep-.chn'
        )
    
    def _parse_quarter_header(self, header_tuple):
        """Parses a messy header tuple from pandas into a clean 'Qx-YYYY' string."""
        # Header can be ('Unnamed: 1_level_0', 'Quý 1-2023')
        header_string = header_tuple[1]
        match = re.search(r'Quý\s*(\d)-(\d{4})', header_string)
        if match:
            quarter, year = match.groups()
            return f"Q{quarter}-{year}"
        return None

    def get_category(self, report_type):
        """Gets the financial statement item names (the first column)."""
        logging.info(f"Fetching categories for report type: {report_type}")
        # Try fetching from the latest quarterly report first
        url = self._build_url(report_type, self.end_year, quarter=1)
        try:
            df_list = pd.read_html(url)
            df_temp = df_list[4] 
            df_category = df_temp.iloc[:, 0]
            if isinstance(df_category.iloc[-1], str) and 'Xem' in df_category.iloc[-1]:
                 df_category = df_category.iloc[:-1]
            return df_category
        except Exception as e:
            logging.warning(f"Could not fetch categories from quarterly URL ({url}). Falling back to yearly. Error: {e}")
            try:
                url_yearly = self._build_url(report_type, self.end_year, quarter=0)
                df_list = pd.read_html(url_yearly)
                df_temp = df_list[4]
                df_category = df_temp.iloc[:, 0]
                if isinstance(df_category.iloc[-1], str) and 'Xem' in df_category.iloc[-1]:
                    df_category = df_category.iloc[:-1]
                return df_category
            except Exception as e2:
                logging.error(f"Fallback to yearly URL also failed for categories. Error: {e2}")
                return None


    def get_stat(self, report_type, period='yearly'):
        """
        Fetches the financial statistics for either yearly or quarterly periods.
        
        Parameters:
            report_type (str): Type of report ('incsta', 'bsheet', 'cashflow').
            period (str): 'yearly' or 'quarterly'.
        """
        if period == 'yearly':
            return self._get_yearly_stat(report_type)
        elif period == 'quarterly':
            return self._get_quarterly_stat(report_type)
        else:
            raise ValueError("Period must be either 'yearly' or 'quarterly'.")

    def _get_yearly_stat(self, report_type):
        """Handles fetching of yearly data."""
        logging.info(f"Fetching YEARLY data for {self.company_name} from {self.start_year} to {self.end_year}.")
        result = pd.DataFrame()
        for year in range(self.end_year, self.start_year - 1, -1):
            url = self._build_url(report_type, year, quarter=0)
            try:
                web_data = pd.read_html(url)
                data_column = web_data[4].iloc[:, 4]
                if not data_column.isna().all():
                    result[year] = data_column
                    logging.info(f"Successfully fetched yearly data for {year}.")
                else:
                    logging.warning(f"No yearly data available for {year}.")
            except Exception as e:
                logging.warning(f"Could not fetch yearly data for {year}. URL: {url} | Error: {e}")
        return result.sort_index(axis=1) # Sort columns by year

    def _get_quarterly_stat(self, report_type):
        """Handles fetching of quarterly data efficiently."""
        logging.info(f"Fetching QUARTERLY data for {self.company_name} from {self.start_year} to {self.end_year}.")
        result = pd.DataFrame()
        collected_quarters = set()
        # Determine the current quarter
        now = datetime.datetime.now()
        current_quarter = (now.month - 1) // 3 + 1

        # Iterate backwards from the most recent possible quarter
        for year in range(self.end_year, self.start_year - 1, -1):
            start_q = current_quarter if year == self.end_year else 4
            for quarter in range(start_q, 0, -1):
                target_col_name = f"Q{quarter}-{year}"
                if target_col_name in collected_quarters:
                    continue
                
                # We need this data, so fetch the page
                logging.info(f"Requesting page for {target_col_name} to get multiple quarters...")
                url = self._build_url(report_type, year, quarter)
                
                try:
                    web_data = pd.read_html(url)[4]
                    # The last row is often a "view more" link, remove it
                    if isinstance(web_data.iloc[-1, 0], str) and 'Xem' in web_data.iloc[-1, 0]:
                        web_data = web_data.iloc[:-1]

                    # Now, parse all columns from this single page
                    break
                    for i, header_tuple in enumerate(web_data.columns[:-1]):
                        if i == 0: continue # Skip the first column (category names)
                        col_name = self._parse_quarter_header(header_tuple)
                        if col_name and col_name not in collected_quarters:
                            result[col_name] = web_data.iloc[:, i]
                            collected_quarters.add(col_name)
                            logging.info(f"  > Found and collected data for {col_name}")

                except Exception as e:
                    logging.warning(f"Could not process page for {target_col_name}. URL: {url} | Error: {e}")
        print(result.head(3))
        return result.sort_index(axis=1) # Sort columns chronologically


    def get_findata(self, report_type, period='yearly', report_format='analysis'):
        """
        Combines categories and stats into a single DataFrame.
        """
        if report_type not in self.report_types:
            raise ValueError(f"report_type must be one of {self.report_types}")

        categories = self.get_category(report_type)
        stats = self.get_stat(report_type, period)
        
        if categories is None or stats.empty:
            logging.error(f"Cannot generate final DataFrame for {report_type} due to missing categories or stats.")
            return pd.DataFrame()

        # Align categories and stats based on index
        result = pd.concat([categories, stats], axis='columns', join='inner')
        result.set_index(0, inplace=True)
        result.index.name = "Financial Statement Item"

        if report_format == 'analysis':
            return result.T
        else:
            return result

    def export_all_reports(self, period='yearly', form='csv', report_format='analysis'):
        """
        Exports all three financial reports to files.
        """
        logging.info(f"Starting export for all reports. Period: {period}, Format: {form}")
        for report_type in self.report_types:
            data = self.get_findata(report_type, period, report_format)
            if data.empty:
                logging.warning(f"Skipping export for {report_type} as no data was found.")
                continue

            file_name = f"{self.company_name}_{report_type}_{period}"
            if form == 'csv':
                file_path = file_name + '.csv'
                data.to_csv(file_path)
                logging.info(f"Successfully saved data to {file_path}")
            elif form == 'xlsx':
                file_path = file_name + '.xlsx'
                data.to_excel(file_path)
                logging.info(f"Successfully saved data to {file_path}")
            else:
                logging.error(f"Unsupported form: '{form}'. Please choose 'csv' or 'xlsx'.")
        logging.info("Export process finished.")




# Initialize the class for the company 'FPT'

# --- Example 1: Get QUARTERLY Income Statement ---
# print("\n--- Fetching Quarterly Income Statement for FPT ---")
# quarterly_income_statement = fpt_fin.get_findata(
#     report_type='bsheet',
#     period='quarterly',
#     report_format='analysis'
# )
# print(quarterly_income_statement.head())


# --- Example 2: Get YEARLY Balance Sheet ---
fpt_fin = FinanceStat(company_name='vnm', start_year=2020)
print("\n--- Fetching Yearly Balance Sheet for FPT ---")
yearly_balance_sheet = fpt_fin.get_findata(
    report_type='bsheet',
    period='yearly',
    report_format='traditional'
)
yearly_balance_sheet

2025-08-13 10:14:08,297 - INFO - Fetching categories for report type: bsheet



--- Fetching Yearly Balance Sheet for FPT ---


2025-08-13 10:14:08,925 - INFO - Fetching YEARLY data for VNM from 2020 to 2025.
2025-08-13 10:14:10,046 - WARNING - No yearly data available for 2025.
2025-08-13 10:14:10,642 - INFO - Successfully fetched yearly data for 2024.
2025-08-13 10:14:11,321 - INFO - Successfully fetched yearly data for 2023.
2025-08-13 10:14:12,001 - INFO - Successfully fetched yearly data for 2022.
2025-08-13 10:14:12,529 - INFO - Successfully fetched yearly data for 2021.
2025-08-13 10:14:13,140 - INFO - Successfully fetched yearly data for 2020.


,2020,2021,2022,2023,2024
Financial Statement Item,,,,,
TÀI SẢN,NaN,NaN,NaN,NaN,NaN
A- TÀI SẢN NGẮN HẠN,2.966573e+13,3.610991e+13,3.156038e+13,3.593588e+13,3.755365e+13
I. Tiền và các khoản tương đương tiền,2.111243e+12,2.348552e+12,2.299944e+12,2.912027e+12,2.225944e+12
1. Tiền,8.638533e+11,1.187350e+12,1.327430e+12,1.025487e+12,1.877944e+12
2. Các khoản tương đương tiền,1.247390e+12,1.161202e+12,9.725140e+11,1.886540e+12,3.480000e+11
...,...,...,...,...,...
13. Lợi ích cổ đông không kiểm soát,2.349939e+12,2.766835e+12,2.967467e+12,3.328929e+12,3.895583e+12
II. Nguồn kinh phí và quỹ khác,NaN,NaN,NaN,NaN,NaN
1. Nguồn kinh phí,NaN,NaN,NaN,NaN,NaN
